## **Kubernetes Objects: The Categorized Cheat Sheet**

Think of Kubernetes like a **city**: Pods are the **citizens**, and everything else is the **infrastructure** that supports them.

### **🎯 CORE DISTINCTION**

| **Cluster Objects** | **Pod Manifest Config** |
|---------------------|------------------------|
| Created separately | Defined INSIDE pod spec |
| Live independently | Tied to the pod's lifecycle |
| Affect multiple pods | Affect only that container/pod |
| `kubectl create -f object.yaml` | Part of pod/deployment YAML |

---

## **📋 PART 1: CLUSTER OBJECTS (Standalone Resources)**

### **🏗️ Infrastructure & Networking**
| Object | Purpose | Quick Example |
|--------|---------|---------------|
| **Service** | Stable network endpoint for pods | `ClusterIP`, `NodePort`, `LoadBalancer` |
| **EndpointSlice** | Tracks pod IPs for a service | Auto-created, don't touch manually |
| **Ingress** | HTTP/S routing to services | Host/path based routing, TLS |
| **NetworkPolicy** | Firewall rules between pods | Allow/deny traffic at IP/port level |
| **ServiceAccount** | Identity for pods | "Who" the pod is |

### **⚖️ Scaling & Availability**
| Object | Purpose | When to Use |
|--------|---------|-------------|
| **HPA** (HorizontalPodAutoscaler) | Auto-scale replicas based on CPU/memory/custom metrics | Variable load |
| **VPA** (VerticalPodAutoscaler) | Auto-adjust CPU/memory requests | Rightsizing |
| **PDB** (PodDisruptionBudget) | Guarantee min available pods during voluntary disruptions | Stateful apps, quorum |
| **PriorityClass** | Pod priority (which gets evicted first) | Critical vs batch jobs |

### **🔒 Access Control**
| Object | Purpose | Example |
|--------|---------|---------|
| **Role** | Permissions WITHIN a namespace | `create pods`, `get logs` |
| **ClusterRole** | Permissions cluster-wide | `get nodes`, `create persistentvolumes` |
| **RoleBinding** | Bind Role to users/serviceaccounts | "Give team X this Role in namespace Y" |
| **ClusterRoleBinding** | Bind ClusterRole cluster-wide | "Give admin access to whole cluster" |

### **📦 Storage**
| Object | Purpose | Lifecycle |
|--------|---------|-----------|
| **PV** (PersistentVolume) | Storage resource (like a disk) | Cluster admin creates |
| **PVC** (PersistentVolumeClaim) | Request for storage | User creates, pod consumes |
| **StorageClass** | Storage types (fast, slow, SSD) | Defines provisioner, parameters |

### **🚦 Governance & Limits**
| Object | Purpose | Scope |
|--------|---------|-------|
| **LimitRange** | Default/min/max resources per container | Namespace-wide |
| **ResourceQuota** | Total resource limits per namespace | Namespace-wide total |
| **NetworkPolicy** | Traffic rules | Pod-to-pod communication |

### **📊 Special Purpose**
| Object | Purpose | Notes |
|--------|---------|-------|
| **Job/CronJob** | Run-to-completion tasks | Batch processing |
| **Lease** | Distributed locking | Used internally by controllers |
| **Event** | Things that happen in cluster | `kubectl get events` |

---

## **📋 PART 2: POD MANIFEST CONFIG (Inside spec)**

### **🏷️ Pod Metadata**
```yaml
apiVersion: v1
kind: Pod
metadata:
  name: my-pod
  labels:          # 🏷️ Key-value for selection
    app: frontend
    tier: web
    environment: prod
  annotations:     # 📝 Non-identifying metadata
    prometheus.io/scrape: "true"
    sidecar.istio.io/inject: "true"
```

### **⚙️ Container Configuration**
```yaml
spec:
  containers:
  - name: app
    image: nginx:1.21
    
    # 🚦 Probes (Health Checks)
    livenessProbe:      # Is container alive? Restart if fails
      httpGet: path: /healthz
      initialDelaySeconds: 30
    readinessProbe:     # Is container ready to serve traffic?
      httpGet: path: /ready
    startupProbe:       # For slow-starting apps
      httpGet: path: /startup
    
    # 💻 Resources (The famous requests/limits)
    resources:
      requests:  # Minimum guaranteed
        cpu: "250m"      # 1/4 core
        memory: "256Mi"
      limits:    # Maximum allowed
        cpu: "500m"      # 1/2 core
        memory: "512Mi"
    
    # 🔌 Environment
    env:
    - name: DB_HOST
      value: "mysql.prod.svc.cluster.local"
    - name: POD_NAME          # Downward API
      valueFrom:
        fieldRef:
          fieldPath: metadata.name
    
    # 📂 Volume Mounts
    volumeMounts:
    - name: config
      mountPath: /etc/config
      readOnly: true
    - name: data
      mountPath: /var/data
```

### **🎯 Scheduling Controls**
```yaml
spec:
  # Node selection
  nodeSelector:
    disktype: ssd
    gpu: nvidia
  
  # Advanced scheduling
  affinity:
    nodeAffinity:
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 100
        preference:
          matchExpressions:
          - key: topology.kubernetes.io/zone
            operator: In
            values: ["us-west-1a"]
    podAntiAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
      - labelSelector:
          matchExpressions:
          - key: app
            operator: In
            values: ["my-app"]
        topologyKey: kubernetes.io/hostname
  
  # Special nodes access
  tolerations:
  - key: "dedicated"
    operator: "Equal"
    value: "gpu"
    effect: "NoSchedule"
  
  # ⚡ Priority
  priorityClassName: high-priority
```

### **🚀 Pod Behavior**
```yaml
spec:
  # Lifecycle hooks
  containers:
  - name: app
    lifecycle:
      postStart:
        exec:
          command: ["/bin/sh", "-c", "echo Started > /tmp/start"]
      preStop:
        exec:
          command: ["/bin/sh", "-c", "sleep 10 && /drain-connections"]
  
  # Termination
  terminationGracePeriodSeconds: 60
  
  # Restart policy
  restartPolicy: Always  # or OnFailure, Never
  
  # DNS
  dnsPolicy: ClusterFirst
  dnsConfig:
    nameservers:
    - 8.8.8.8
    
  # Security
  securityContext:
    runAsUser: 1000
    fsGroup: 2000
  containers:
  - name: app
    securityContext:
      allowPrivilegeEscalation: false
      capabilities:
        drop: ["ALL"]
```

### **📦 Volumes (defined at pod level)**
```yaml
spec:
  volumes:
  - name: config
    configMap:
      name: app-config
  - name: secrets
    secret:
      secretName: app-secrets
  - name: empty
    emptyDir: {}
  - name: persistent
    persistentVolumeClaim:
      claimName: my-pvc
  - name: host
    hostPath:
      path: /var/log
      type: Directory
```

---

## **📊 QUICK REFERENCE CARD**

### **When to Use What**

| **You want to...** | **Use this OBJECT** | **Or this POD CONFIG** |
|--------------------|---------------------|------------------------|
| Expose pods to network | Service | - |
| Scale based on load | HPA | - |
| Set pod priority | PriorityClass | `priorityClassName` |
| Control pod resources | - | `resources.requests/limits` |
| Ensure pod availability | PDB | - |
| Health check pods | - | `livenessProbe`, `readinessProbe` |
| Store configuration | ConfigMap | `env`, `volumeMounts` |
| Store secrets | Secret | `env`, `volumeMounts` |
| Control pod placement | - | `nodeSelector`, `affinity` |
| Restrict traffic | NetworkPolicy | - |
| Schedule on special nodes | - | `tolerations` |
| Pod identity | ServiceAccount | `serviceAccountName` |
| Run once tasks | Job | - |
| Access metadata | - | Downward API (`fieldRef`) |

---

## **🎤 Interview Soundbite Ready**

> **"Kubernetes has two configuration worlds: cluster objects and pod manifests. Objects like HPAs, PDBs, and Services are standalone resources that manage pods from the outside. Inside pod manifests, I define container-specific config—probes for health, requests/limits for resources, and affinity for placement. The key insight: cluster objects affect many pods and live independently; pod config is embedded and dies with the pod."**

### **Pro Tip for Interviews**
When asked about any Kubernetes concept, quickly categorize it:
- **"Is this a cluster object or pod manifest config?"**
- This shows you understand the architecture, not just syntax